# Prompt Free Generation

上からび～ってなぞるだけ(ctrl+F9)でUIが出てくるようにはする。
ただあえてコードは隠さない。A100の場合は下の2セルうまく変えて。

In [1]:
#T4対応のほいーる。
!pip -qqqq install -U -I --no-deps https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 MB 8.0 MB/s eta 0:00:00


In [2]:
#T4/A100対応xformersのインストール from https://github.com/brian6091/xformers-wheels/releases/tag/0.0.15.dev0%2B4c06c79
#A100で使えてうれしいが、torchのバージョン変えるので時間がかかる
#!pip install https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl

In [3]:
#必要ライブラリ
!pip install gradio diffusers accelerate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.3/716.3 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 K

In [4]:
#wdtaggerとpfg重みを一気にダウンロード
!sudo apt update
!sudo apt install git-lfg
!git lfs install
!git clone https://huggingface.co/SmilingWolf/wd-v1-4-vit-tagger-v2
!wget https://huggingface.co/furusu/PFG/resolve/main/pfg-wd14-n10.pt

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [10

In [1]:
#なんか知らんがこれしないとtensorflowがVRAM全部食べて死ぬ
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True


In [2]:
####このコードは@kohya-ss氏のLoRANetwork定義を参考にしています。感謝いたします。https://github.com/kohya-ss/sd-scripts/blob/main/networks/lora.py

import torch
import math
import os
import itertools

UNET_TARGET_REPLACE_MODULE = ["Transformer2DModel"] #CrossAttentionでいいかもしれないがまあいいや

PREFIX_UNET = 'pfg'

    
#Network
class PFGNetwork(torch.nn.Module):
    def __init__(self, unet, input_size:int = 768, cross_attention_dim:int = 1024, num_tokens:int = 1) -> None:
        super().__init__()
        self.input_size = input_size
        self.cross_attention_dim = cross_attention_dim
        self.num_tokens = num_tokens
        
        #inputを入れる
        self.input = None
        
        #input_dim -> token_dim * num_tokens
        self.pfg_linear = torch.nn.Linear(self.input_size, cross_attention_dim * num_tokens)
        #unetのforward書き換え
        self._hook_forwards(unet, UNET_TARGET_REPLACE_MODULE)
        self.requires_grad_(True)
        
    #見づらいのでメソッドにしちゃう
    #名前変えるのめんどいからloraのまま
    def _hook_forwards(self, root_module: torch.nn.Module, target_replace_modules) -> list:
        count = 0
        for name, module in root_module.named_modules():
            if module.__class__.__name__ in target_replace_modules:
                for child_name, child_module in module.named_modules():
                    if child_module.__class__.__name__ == "Linear":
                        #to_k, to_vのみ v1は768, v2は1024
                        if child_module.in_features == self.cross_attention_dim:
                            count += 1
                            child_module.forward = self._hook_forward(child_module)
        print(f"create PFG for U-Net: {count} modules.")
    
    #forward書き換え
    def _hook_forward(self, org_module):
        prev_forward = org_module.forward
        def forward(x):
            #(b,1,dim) -> #(b,1,in_dim * tokens) 
            c = self.pfg_linear(self.input)
            
            #(b,1,dim) -> #(b,tokens,in_dim) 
            c = c.reshape(-1,self.num_tokens,self.cross_attention_dim)
            
            #生成時は違う場合が多い、その場合は(uncond+cond)*batchのため2分の1倍する。
            if x.shape[0] != c.shape[0]:
                c = c.repeat(x.shape[0] // 2,1,1)
                
            #concatenate (b,N,in_dim) + (b,tokens,in_dim)
            if self.input.shape[0] == x.shape[0]:
                x = torch.cat([x,c],dim = 1)
            else: #CFG対応：diffusers pipelineでは(uncond, cond)
                x_uncond, x_cond = x.chunk(2)
                x_uncond = torch.cat([x_uncond,x_uncond[:,-1:,:].repeat(1,self.num_tokens,1)],dim=1) #EOSをコピー（これでいいのか知らん）
                x_cond = torch.cat([x_cond,c],dim = 1) 
                x = torch.cat([x_uncond,x_cond])

            return prev_forward(x)
        return forward
    
    def prepare_optimizer_params(self, unet_lr):
        self.requires_grad_(True)
        param_data = {'params': self.parameters()}
        if unet_lr is not None:
            param_data['lr'] = unet_lr
        
        return [param_data]
            

    def save_weights(self, file, dtype=None):
        state_dict = self.state_dict()

        if dtype is not None:
            for key in list(state_dict.keys()):
                v = state_dict[key]
                v = v.detach().clone().to("cpu").to(dtype)
                state_dict[key] = v

        if os.path.splitext(file)[1] == '.safetensors':
            from safetensors.torch import save_file
            save_file(state_dict, file)
        else:
            torch.save(state_dict, file)
    
    def set_input(self,input_tensor:torch.Tensor) -> None:
        self.input = input_tensor

In [3]:
#WD14 taggerモデルのロードとutils
from tensorflow.keras.models import load_model, Model
model = load_model("wd-v1-4-vit-tagger-v2")
model = Model(model.layers[0].input, model.layers[-3].output) #最終層手前のプーリング層の出力を使う

# DanBooru IMage Utility functions
#このコードどこを引用とすればいいか分かんない・・・

import cv2
import numpy as np
from PIL import Image

#pil -> cv2に改変
def smart_imread(img:Image, flag=cv2.IMREAD_UNCHANGED):
    img = img.convert("RGB")
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    return img

def smart_24bit(img):
    if img.dtype is np.dtype(np.uint16):
        img = (img / 257).astype(np.uint8)

    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    elif img.shape[2] == 4:
        trans_mask = img[:, :, 3] == 0
        img[trans_mask] = [255, 255, 255, 255]
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

def make_square(img, target_size):
    old_size = img.shape[:2]
    desired_size = max(old_size)
    desired_size = max(desired_size, target_size)

    delta_w = desired_size - old_size[1]
    delta_h = desired_size - old_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [255, 255, 255]
    new_im = cv2.copyMakeBorder(
        img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color
    )
    return new_im

def smart_resize(img, size):
    # Assumes the image has already gone through make_square
    if img.shape[0] > size:
        img = cv2.resize(img, (size, size), interpolation=cv2.INTER_AREA)
    elif img.shape[0] < size:
        img = cv2.resize(img, (size, size), interpolation=cv2.INTER_CUBIC)
    return img

In [4]:
#wdtaggerの推論

import torch
from PIL import Image
def infer(img:Image):
    img = smart_imread(img)
    img = smart_24bit(img)
    img = make_square(img, 448)
    img = smart_resize(img, 448)
    img = img.astype(np.float32)
    probs = model(np.array([img]), training=False)
    return torch.tensor(probs.numpy()).unsqueeze(0)

In [5]:
#学習したモデルはこれなのだが、SD2系の他のモデルでももしかしたらうまくいくかもしれない。
model_id = "hakurei/waifu-diffusion"

In [7]:
#Stable diffusion モデルのロード
#かすたむすくりぷとなるものをつかう
from diffusers import DiffusionPipeline
device = "cuda"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None,
    custom_pipeline="lpw_stable_diffusion", #なんか知らんがwebui風のプロンプト強調()[]ができるらすい
).to(device)

try:
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_vae_slicing()
    print("apply xformers for unet !!!")
except:
    print("cant apply xformers. using normal unet !!!")


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers_modules.git.lpw_stable_diffusion.StableDiffusionLongPromptWeightingPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


apply xformers for unet !!!


In [8]:
pfg = PFGNetwork(pipe.unet, 768, 1024, 10)
pfg.load_state_dict(torch.load("pfg-wd14-n10.pt"))
pfg.requires_grad_(False)
pfg.to(device, dtype = torch.float16)
print("loaded")

create PFG for U-Net: 32 modules.
loaded


In [10]:
import gradio as gr
from datetime import datetime


def generate(image,prompt,negative_prompt,batch_size,image_scale,width,height,steps,guidance_scale):
    hidden_states = infer(image).to(device)
    pfg.set_input(hidden_states * image_scale)
    with torch.autocast("cuda"):
        images = pipe(prompt = [prompt]*batch_size,
                      width = width,
                      height = height,
                      guidance_scale=guidance_scale,
                      num_inference_steps=steps,
                      negative_prompt = [negative_prompt]*batch_size,
                      ).images
    return images

demo = gr.Interface(
    generate,
    [gr.Image(type="pil"),
     gr.Textbox(value="illustration of"),
     gr.Textbox(value="worst quality, low quality, medium quality, deleted, lowres, comic, bad anatomy,bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry"),
     gr.Slider(1,32,value=1,step=1),
     gr.Slider(0,2,value=1,step=0.01,label="pfgの強弱"),
     gr.Slider(256,1280,value=640,step=64),
     gr.Slider(256,1280,value=896,step=64),
     gr.Slider(0,50,value=50,step=1),
     gr.Slider(0,30,value=12,step=0.5)],
    [gr.Gallery(label='Output', show_label=False, elem_id="gallery").style(grid=[2], height="auto")],
    title = "タイトル",
    description = "説明",
    allow_flagging='never'
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9e6a9387-5599-4fa8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
